# Lab 09: AgentCore 리소스 정리

이 노트북은 **이 프로젝트에서 생성한 AgentCore 리소스만** 정리합니다. 다른 프로젝트의 리소스와 CloudFormation 인프라는 유지됩니다.

## 정리 대상

| 리소스 | 삭제 여부 | 설명 |
|--------|----------|------|
| AgentCore Runtime | ✅ 패턴 매칭 | 프로젝트 패턴과 일치하는 런타임만 삭제 |
| AgentCore Memory | ✅ 패턴 매칭 | 프로젝트 패턴과 일치하는 메모리만 삭제 |
| AgentCore Gateway | ✅ 패턴 매칭 | 프로젝트 패턴과 일치하는 게이트웨이만 삭제 |
| CloudWatch Logs | ✅ 패턴 매칭 | 프로젝트 패턴과 일치하는 로그 그룹만 삭제 |
| ECR Repository | ✅ 패턴 매칭 | 프로젝트 패턴과 일치하는 ECR만 삭제 |
| SSM Parameters | ✅ 삭제 | AgentCore 관련 파라미터 |
| CloudFormation | ❌ 유지 | Cognito, IAM 역할 등 |
| Secrets Manager | ❌ 유지 | Cognito 인증 정보 |

## 용도

- 노트북 재실행을 위한 AgentCore 리소스 초기화
- 인프라는 유지하면서 에이전트만 재배포
- 비용 절감을 위한 런타임 정리

## 완전 삭제 (Step 11)

인프라까지 모두 삭제하려면 Step 11의 주석을 해제하세요.

## Step 1: 프로젝트 필터 설정

**이 프로젝트에서 생성한 리소스만 삭제합니다.** 다른 프로젝트의 리소스는 유지됩니다.

아래 패턴을 수정하여 삭제 대상을 조정할 수 있습니다.

In [7]:
# ============================================================
# 프로젝트 필터 설정 - 필요시 수정하세요
# ============================================================

# Runtime 이름 패턴 (이 패턴으로 시작하는 런타임만 삭제)
RUNTIME_PATTERNS = [
    "lab5_",
    "lab6_",
    "lab7_",
    "ecommerce_test_",
]

# Memory 이름 패턴 (이 패턴을 포함하는 메모리만 삭제)
MEMORY_PATTERNS = [
    "EcommerceCustomerMemory",
]

# Gateway 이름 패턴 (이 패턴으로 시작하는 게이트웨이만 삭제)
GATEWAY_PATTERNS = [
    "ecommerce-gw",
]

# CloudWatch 로그 그룹 패턴 (이 패턴을 포함하는 로그만 삭제)
LOG_PATTERNS = [
    "lab5_",
    "lab6_",
    "lab7_",
    "ecommerce_test_",
]

# ECR 리포지토리 패턴 (이 패턴을 포함하는 ECR만 삭제)
ECR_PATTERNS = [
    "lab5_",
    "lab6_",
    "lab7_",
    "ecommerce_test_",
]

# 모든 리소스 삭제 (True로 설정하면 필터 무시하고 모든 AgentCore 리소스 삭제)
DELETE_ALL = False

print("📋 프로젝트 필터 설정 완료")
print(f"   DELETE_ALL: {DELETE_ALL}")
if not DELETE_ALL:
    print(f"   Runtime 패턴: {RUNTIME_PATTERNS}")
    print(f"   Memory 패턴: {MEMORY_PATTERNS}")
    print(f"   Gateway 패턴: {GATEWAY_PATTERNS}")
    print(f"   Log 패턴: {LOG_PATTERNS}")
    print(f"   ECR 패턴: {ECR_PATTERNS}")

📋 프로젝트 필터 설정 완료
   DELETE_ALL: False
   Runtime 패턴: ['lab5_', 'lab6_', 'lab7_', 'ecommerce_test_']
   Memory 패턴: ['EcommerceCustomerMemory']
   Gateway 패턴: ['ecommerce-gw']
   Log 패턴: ['lab5_', 'lab6_', 'lab7_', 'ecommerce_test_']
   ECR 패턴: ['lab5_', 'lab6_', 'lab7_', 'ecommerce_test_']


## Step 2: 환경 설정

In [8]:
import boto3
import os
import json
import time
from botocore.exceptions import ClientError

# AWS 클라이언트 설정
REGION = "us-east-1"
session = boto3.Session(region_name=REGION)
ssm_client = session.client('ssm')
logs_client = session.client('logs')
ecr_client = session.client('ecr')
sts_client = session.client('sts')
secrets_client = session.client('secretsmanager')
cloudformation_client = session.client('cloudformation')
iam_client = session.client('iam')

# AgentCore 클라이언트
try:
    agentcore_client = session.client('bedrock-agentcore-control')
    print("✅ bedrock-agentcore-control 클라이언트 설정 완료")
except Exception as e:
    print(f"⚠️ bedrock-agentcore-control 클라이언트 설정 실패: {e}")
    agentcore_client = None

ACCOUNT_ID = sts_client.get_caller_identity()['Account']

print(f"🌍 리전: {REGION}")
print(f"🔑 계정: {ACCOUNT_ID}")
print("\n🎯 AgentCore 리소스만 정리합니다 (인프라 유지)")

✅ bedrock-agentcore-control 클라이언트 설정 완료
🌍 리전: us-east-1
🔑 계정: 057716757052

🎯 AgentCore 리소스만 정리합니다 (인프라 유지)


## Step 3: AgentCore Runtime 정리

프로젝트 패턴과 일치하는 Runtime만 삭제합니다.

In [9]:
print("🚀 AgentCore Runtime 정리 시작...")

deleted_runtimes = []
skipped_runtimes = []
failed_runtimes = []

def matches_pattern(name, patterns):
    """이름이 패턴 목록 중 하나와 일치하는지 확인"""
    return any(name.startswith(p) or p in name for p in patterns)

if agentcore_client:
    try:
        response = agentcore_client.list_agent_runtimes()
        
        for runtime in response.get('agentRuntimes', []):
            runtime_id = runtime['agentRuntimeId']
            runtime_name = runtime.get('agentRuntimeName', 'N/A')
            status = runtime.get('status', 'UNKNOWN')
            
            # 패턴 필터링 (DELETE_ALL=True면 모두 삭제)
            if DELETE_ALL or matches_pattern(runtime_name, RUNTIME_PATTERNS):
                print(f"  📋 삭제 대상: {runtime_name} ({runtime_id}) - {status}")
                
                try:
                    agentcore_client.delete_agent_runtime(agentRuntimeId=runtime_id)
                    deleted_runtimes.append(runtime_name)
                    print(f"    ✅ 삭제 완료: {runtime_name}")
                except Exception as e:
                    failed_runtimes.append((runtime_name, str(e)))
                    print(f"    ❌ 삭제 실패: {e}")
            else:
                skipped_runtimes.append(runtime_name)
                print(f"  ⏭️ 건너뜀: {runtime_name} (패턴 불일치)")
                    
    except Exception as e:
        print(f"❌ 런타임 목록 조회 실패: {e}")
else:
    print("⚠️ AgentCore 클라이언트 없음 - 런타임 정리 건너뜀")

print(f"\n📊 Runtime 정리 결과: {len(deleted_runtimes)}개 삭제, {len(skipped_runtimes)}개 건너뜀, {len(failed_runtimes)}개 실패")

🚀 AgentCore Runtime 정리 시작...
  📋 삭제 대상: lab5_streaming_dcd88b77 (lab5_streaming_dcd88b77-hlPVFZA25n) - DELETING


    ❌ 삭제 실패: An error occurred (ConflictException) when calling the DeleteAgentRuntime operation: The agent is currently being modified by another operation. Current status: DELETING. Wait and try again.

📊 Runtime 정리 결과: 0개 삭제, 0개 건너뜀, 1개 실패


## Step 4: AgentCore Memory 정리

프로젝트 패턴과 일치하는 Memory만 삭제합니다.

In [10]:
print("🧠 AgentCore Memory 정리 시작...")

deleted_memories = []
skipped_memories = []
failed_memories = []

if agentcore_client:
    try:
        response = agentcore_client.list_memories()
        
        for memory in response.get('memories', []):
            memory_id = memory['id']
            memory_name = memory_id.split('-')[0] if '-' in memory_id else memory_id
            
            # 패턴 필터링 (DELETE_ALL=True면 모두 삭제)
            if DELETE_ALL or matches_pattern(memory_id, MEMORY_PATTERNS):
                print(f"  📋 삭제 대상: {memory_id}")
                
                try:
                    agentcore_client.delete_memory(memoryId=memory_id)
                    deleted_memories.append(memory_id)
                    print(f"    ✅ 삭제 완료: {memory_id}")
                except Exception as e:
                    failed_memories.append((memory_id, str(e)))
                    print(f"    ❌ 삭제 실패: {e}")
            else:
                skipped_memories.append(memory_id)
                print(f"  ⏭️ 건너뜀: {memory_id} (패턴 불일치)")
                    
    except Exception as e:
        print(f"❌ 메모리 목록 조회 실패: {e}")
else:
    print("⚠️ AgentCore 클라이언트 없음 - 메모리 정리 건너뜀")

print(f"\n📊 Memory 정리 결과: {len(deleted_memories)}개 삭제, {len(skipped_memories)}개 건너뜀, {len(failed_memories)}개 실패")

🧠 AgentCore Memory 정리 시작...

📊 Memory 정리 결과: 0개 삭제, 0개 건너뜀, 0개 실패


## Step 5: AgentCore Gateway 정리

프로젝트 패턴과 일치하는 Gateway만 삭제합니다.

In [13]:
print("🌐 AgentCore Gateway 정리 시작...")

deleted_gateways = []
skipped_gateways = []
failed_gateways = []

if agentcore_client:
    try:
        response = agentcore_client.list_gateways()
        
        for gateway in response.get('items', []):
            gateway_id = gateway['gatewayId']
            gateway_name = gateway.get('name', 'N/A')
            
            # 패턴 필터링 (DELETE_ALL=True면 모두 삭제)
            if DELETE_ALL or matches_pattern(gateway_name, GATEWAY_PATTERNS):
                print(f"  📋 삭제 대상: {gateway_name} ({gateway_id})")
                
                # 먼저 게이트웨이 타겟들 삭제
                targets_deleted = 0
                try:
                    targets_response = agentcore_client.list_gateway_targets(
                        gatewayIdentifier=gateway_id
                    )
                    
                    for target in targets_response.get('items', []):
                        target_id = target['targetId']
                        print(f"    🎯 타겟 삭제: {target_id}")
                        agentcore_client.delete_gateway_target(
                            gatewayIdentifier=gateway_id,
                            targetId=target_id
                        )
                        targets_deleted += 1
                        print(f"      ✅ 타겟 삭제 완료: {target_id}")
                except Exception as e:
                    print(f"    ⚠️ 타겟 삭제 중 오류: {e}")
                
                # 타겟 삭제 후 잠시 대기 (비동기 처리 완료 대기)
                if targets_deleted > 0:
                    print(f"    ⏳ 타겟 삭제 완료 대기 중... (3초)")
                    time.sleep(3)
                
                # 게이트웨이 삭제
                try:
                    agentcore_client.delete_gateway(gatewayIdentifier=gateway_id)
                    deleted_gateways.append(gateway_name)
                    print(f"    ✅ 삭제 완료: {gateway_name}")
                except Exception as e:
                    failed_gateways.append((gateway_name, str(e)))
                    print(f"    ❌ 삭제 실패: {e}")
            else:
                skipped_gateways.append(gateway_name)
                print(f"  ⏭️ 건너뜀: {gateway_name} (패턴 불일치)")
                    
    except Exception as e:
        print(f"❌ 게이트웨이 목록 조회 실패: {e}")
else:
    print("⚠️ AgentCore 클라이언트 없음 - 게이트웨이 정리 건너뜀")

print(f"\n📊 Gateway 정리 결과: {len(deleted_gateways)}개 삭제, {len(skipped_gateways)}개 건너뜀, {len(failed_gateways)}개 실패")

🌐 AgentCore Gateway 정리 시작...
  ⏭️ 건너뜀: customersupport-gw (패턴 불일치)
  ⏭️ 건너뜀: DemoGWOpenAPIAPIKeyNasaOAI (패턴 불일치)
  📋 삭제 대상: ecommerce-gw (ecommerce-gw-5m5bwlsqkk)
    ✅ 삭제 완료: ecommerce-gw
  ⏭️ 건너뜀: GW-Insurance-Underwriting (패턴 불일치)
  ⏭️ 건너뜀: TestGWforLambda (패턴 불일치)

📊 Gateway 정리 결과: 1개 삭제, 4개 건너뜀, 0개 실패


## Step 6: CloudWatch 로그 그룹 정리

프로젝트 패턴과 일치하는 CloudWatch 로그 그룹만 삭제합니다.

In [14]:
print("📊 CloudWatch 로그 그룹 정리 시작...")

deleted_logs = []
skipped_logs = []
failed_logs = []

# AgentCore 관련 기본 패턴
base_patterns = [
    "/aws/bedrock-agentcore/",
    "/aws/codebuild/bedrock-agentcore-",
]

try:
    paginator = logs_client.get_paginator('describe_log_groups')
    
    for page in paginator.paginate():
        for log_group in page['logGroups']:
            log_group_name = log_group['logGroupName']
            
            # AgentCore 관련 로그 그룹인지 확인
            is_agentcore_log = any(pattern in log_group_name for pattern in base_patterns)
            
            if is_agentcore_log:
                # 프로젝트 패턴과 일치하는지 확인 (DELETE_ALL=True면 모두 삭제)
                if DELETE_ALL or any(pattern in log_group_name for pattern in LOG_PATTERNS):
                    print(f"  📋 삭제 대상: {log_group_name}")
                    
                    try:
                        logs_client.delete_log_group(logGroupName=log_group_name)
                        deleted_logs.append(log_group_name)
                        print(f"    ✅ 삭제 완료")
                    except Exception as e:
                        failed_logs.append((log_group_name, str(e)))
                        print(f"    ❌ 삭제 실패: {e}")
                else:
                    skipped_logs.append(log_group_name)
                    
except Exception as e:
    print(f"❌ 로그 그룹 목록 조회 실패: {e}")

print(f"\n📊 CloudWatch 정리 결과: {len(deleted_logs)}개 삭제, {len(skipped_logs)}개 건너뜀, {len(failed_logs)}개 실패")

📊 CloudWatch 로그 그룹 정리 시작...
  📋 삭제 대상: /aws/bedrock-agentcore/runtimes/ecommerce_test_03921b22-8q5UcT8cQW-DEFAULT
    ✅ 삭제 완료
  📋 삭제 대상: /aws/bedrock-agentcore/runtimes/ecommerce_test_6d428c98-RGw0v722O2-DEFAULT
    ✅ 삭제 완료
  📋 삭제 대상: /aws/bedrock-agentcore/runtimes/ecommerce_test_7b29c478-522N0a4bUT-DEFAULT
    ✅ 삭제 완료
  📋 삭제 대상: /aws/bedrock-agentcore/runtimes/ecommerce_test_bb595171-XHal612EWK-DEFAULT
    ✅ 삭제 완료
  📋 삭제 대상: /aws/bedrock-agentcore/runtimes/ecommerce_test_f09cf5ae-NJDqxE6cry-DEFAULT
    ✅ 삭제 완료
  📋 삭제 대상: /aws/bedrock-agentcore/runtimes/ecommerce_test_fd9f4109-HCL2j82kW4-DEFAULT
    ✅ 삭제 완료
  📋 삭제 대상: /aws/bedrock-agentcore/runtimes/lab5_streaming_40da8868-fbN7zVGWDM-DEFAULT
    ✅ 삭제 완료
  📋 삭제 대상: /aws/bedrock-agentcore/runtimes/lab5_streaming_428cf920-QezecUDfGC-DEFAULT
    ✅ 삭제 완료
  📋 삭제 대상: /aws/bedrock-agentcore/runtimes/lab5_streaming_48dbc433-FlTOquGkpU-DEFAULT
    ✅ 삭제 완료
  📋 삭제 대상: /aws/bedrock-agentcore/runtimes/lab5_streaming_600e9f53-hrhtYR7igp-DEFAULT
    

## Step 7: ECR 리포지토리 정리

프로젝트 패턴과 일치하는 ECR 리포지토리만 삭제합니다.

In [15]:
print("📦 ECR 리포지토리 정리 시작...")

deleted_repos = []
skipped_repos = []
failed_repos = []

# AgentCore ECR 기본 패턴
ecr_base_pattern = "bedrock-agentcore-"

try:
    paginator = ecr_client.get_paginator('describe_repositories')
    
    for page in paginator.paginate():
        for repo in page['repositories']:
            repo_name = repo['repositoryName']
            
            # AgentCore ECR인지 확인
            if ecr_base_pattern in repo_name:
                # 프로젝트 패턴과 일치하는지 확인 (DELETE_ALL=True면 모두 삭제)
                if DELETE_ALL or any(pattern in repo_name for pattern in ECR_PATTERNS):
                    print(f"  📋 삭제 대상: {repo_name}")
                    
                    try:
                        ecr_client.delete_repository(
                            repositoryName=repo_name,
                            force=True
                        )
                        deleted_repos.append(repo_name)
                        print(f"    ✅ 삭제 완료")
                    except Exception as e:
                        failed_repos.append((repo_name, str(e)))
                        print(f"    ❌ 삭제 실패: {e}")
                else:
                    skipped_repos.append(repo_name)
                    
except Exception as e:
    print(f"❌ ECR 목록 조회 실패: {e}")

print(f"\n📊 ECR 정리 결과: {len(deleted_repos)}개 삭제, {len(skipped_repos)}개 건너뜀, {len(failed_repos)}개 실패")

📦 ECR 리포지토리 정리 시작...
  📋 삭제 대상: bedrock-agentcore-lab5_streaming_600e9f53
    ✅ 삭제 완료
  📋 삭제 대상: bedrock-agentcore-lab5_streaming_40da8868
    ✅ 삭제 완료
  📋 삭제 대상: bedrock-agentcore-ecommerce_test_7b29c478
    ✅ 삭제 완료
  📋 삭제 대상: bedrock-agentcore-lab5_streaming_dcd88b77
    ✅ 삭제 완료
  📋 삭제 대상: bedrock-agentcore-lab6_langfuse_e9572f4e
    ✅ 삭제 완료
  📋 삭제 대상: bedrock-agentcore-ecommerce_test_16acd98b
    ✅ 삭제 완료
  📋 삭제 대상: bedrock-agentcore-lab5_streaming_82b595a4
    ✅ 삭제 완료
  📋 삭제 대상: bedrock-agentcore-lab5_streaming_48dbc433
    ✅ 삭제 완료
  📋 삭제 대상: bedrock-agentcore-lab5_test_4d90f142
    ✅ 삭제 완료
  📋 삭제 대상: bedrock-agentcore-lab7_dual_obs_01846bc9
    ✅ 삭제 완료
  📋 삭제 대상: bedrock-agentcore-lab7_dual_obs_16e3b4be
    ✅ 삭제 완료
  📋 삭제 대상: bedrock-agentcore-lab5_streaming_428cf920
    ✅ 삭제 완료
  📋 삭제 대상: bedrock-agentcore-lab6_langfuse_01fb719f
    ✅ 삭제 완료
  📋 삭제 대상: bedrock-agentcore-lab7_dual_89e4559b
    ✅ 삭제 완료

📊 ECR 정리 결과: 14개 삭제, 5개 건너뜀, 0개 실패


## Step 8: SSM 파라미터 정리 (AgentCore 관련만)

AgentCore 관련 SSM 파라미터를 삭제합니다. 인프라 관련 파라미터는 유지됩니다.

In [16]:
print("⚙️ SSM 파라미터 정리 시작 (AgentCore 관련만)...")

deleted_params = []
failed_params = []

# AgentCore 관련 SSM 파라미터만 삭제 (인프라 관련은 유지)
agentcore_params = [
    "/app/ecommerce/agentcore/memory_id",
    "/app/ecommerce/agentcore/memory_arn",
    "/app/ecommerce/agentcore/runtime_arn",
    "/app/ecommerce/agentcore/runtime_id",
    "/app/ecommerce/agentcore/gateway_id",
    "/app/ecommerce/agentcore/gateway_arn",
]

for param_name in agentcore_params:
    try:
        ssm_client.delete_parameter(Name=param_name)
        deleted_params.append(param_name)
        print(f"  ✅ 삭제: {param_name}")
    except ssm_client.exceptions.ParameterNotFound:
        print(f"  ⚠️ 없음: {param_name}")
    except Exception as e:
        failed_params.append((param_name, str(e)))
        print(f"  ❌ 실패: {param_name} - {e}")

print(f"\n📊 SSM 정리 결과: {len(deleted_params)}개 삭제, {len(failed_params)}개 실패")
print("\n🔒 유지됨: /app/ecommerce/agentcore/runtime_iam_role (인프라)")

⚙️ SSM 파라미터 정리 시작 (AgentCore 관련만)...
  ✅ 삭제: /app/ecommerce/agentcore/memory_id
  ⚠️ 없음: /app/ecommerce/agentcore/memory_arn
  ✅ 삭제: /app/ecommerce/agentcore/runtime_arn
  ⚠️ 없음: /app/ecommerce/agentcore/runtime_id
  ✅ 삭제: /app/ecommerce/agentcore/gateway_id
  ⚠️ 없음: /app/ecommerce/agentcore/gateway_arn

📊 SSM 정리 결과: 3개 삭제, 0개 실패

🔒 유지됨: /app/ecommerce/agentcore/runtime_iam_role (인프라)


## Step 9: 로컬 설정 파일 정리

노트북 폴더의 `.bedrock_agentcore.yaml` 파일들을 삭제합니다.

In [17]:
print("📁 로컬 설정 파일 정리 시작...")

import glob

deleted_files = []

# .bedrock_agentcore.yaml 파일 찾기
config_files = glob.glob("**/.bedrock_agentcore.yaml", recursive=True)
config_files += glob.glob("../**/.bedrock_agentcore.yaml", recursive=True)

for config_file in config_files:
    try:
        os.remove(config_file)
        deleted_files.append(config_file)
        print(f"  ✅ 삭제: {config_file}")
    except Exception as e:
        print(f"  ❌ 실패: {config_file} - {e}")

if not config_files:
    print("  ⚠️ 설정 파일 없음")

print(f"\n📊 로컬 파일 정리 결과: {len(deleted_files)}개 삭제")

📁 로컬 설정 파일 정리 시작...
  ✅ 삭제: lab-06-agentcore-observability-langfuse/.bedrock_agentcore.yaml
  ❌ 실패: ../notebooks/lab-06-agentcore-observability-langfuse/.bedrock_agentcore.yaml - [Errno 2] No such file or directory: '../notebooks/lab-06-agentcore-observability-langfuse/.bedrock_agentcore.yaml'

📊 로컬 파일 정리 결과: 1개 삭제


## Step 10: 정리 결과 요약

In [18]:
print("\n" + "=" * 70)
print("📋 AgentCore 리소스 정리 결과 요약")
print("=" * 70)

print(f"\n🚀 Runtime: {len(deleted_runtimes)}개 삭제, {len(skipped_runtimes)}개 건너뜀")
for name in deleted_runtimes:
    print(f"   - {name}")

print(f"\n🧠 Memory: {len(deleted_memories)}개 삭제, {len(skipped_memories)}개 건너뜀")
for name in deleted_memories:
    print(f"   - {name}")

print(f"\n🌐 Gateway: {len(deleted_gateways)}개 삭제, {len(skipped_gateways)}개 건너뜀")
for name in deleted_gateways:
    print(f"   - {name}")

print(f"\n📊 CloudWatch Logs: {len(deleted_logs)}개 삭제, {len(skipped_logs)}개 건너뜀")
print(f"\n📦 ECR Repositories: {len(deleted_repos)}개 삭제, {len(skipped_repos)}개 건너뜀")
print(f"\n⚙️ SSM Parameters: {len(deleted_params)}개 삭제")
print(f"\n📁 Local Files: {len(deleted_files)}개 삭제")

print("\n" + "=" * 70)
print("✅ AgentCore 정리 완료!")
print("=" * 70)

print("\n🔒 유지된 인프라 리소스:")
print("   - CloudFormation 스택 (Cognito, IAM 역할)")
print("   - Secrets Manager (ecommerce_customer_support_agent)")
print("   - SSM 파라미터 (/app/ecommerce/agentcore/runtime_iam_role)")

print("\n🎯 다음 단계:")
print("   - Lab 01부터 다시 실행하여 새 에이전트 생성 가능")
print("   - 인프라가 유지되어 인증/권한 설정 재사용 가능")
print("   - 완전 삭제가 필요하면 Step 11 실행")


📋 AgentCore 리소스 정리 결과 요약

🚀 Runtime: 0개 삭제, 0개 건너뜀

🧠 Memory: 0개 삭제, 0개 건너뜀

🌐 Gateway: 1개 삭제, 4개 건너뜀
   - ecommerce-gw

📊 CloudWatch Logs: 35개 삭제, 86개 건너뜀

📦 ECR Repositories: 14개 삭제, 5개 건너뜀

⚙️ SSM Parameters: 3개 삭제

📁 Local Files: 1개 삭제

✅ AgentCore 정리 완료!

🔒 유지된 인프라 리소스:
   - CloudFormation 스택 (Cognito, IAM 역할)
   - Secrets Manager (ecommerce_customer_support_agent)
   - SSM 파라미터 (/app/ecommerce/agentcore/runtime_iam_role)

🎯 다음 단계:
   - Lab 01부터 다시 실행하여 새 에이전트 생성 가능
   - 인프라가 유지되어 인증/권한 설정 재사용 가능
   - 완전 삭제가 필요하면 Step 11 실행


---

## Step 11: 인프라 완전 삭제 (선택사항)

**주의**: 이 셀을 실행하면 CloudFormation으로 생성한 모든 인프라가 삭제됩니다.
- Cognito User Pool
- IAM 역할
- Secrets Manager
- 인프라 관련 SSM 파라미터

**완전히 정리하려면 아래 셀의 주석을 해제하고 실행하세요.**

In [ ]:
# ============================================================
# 인프라 완전 삭제 - 주석 해제 후 실행
# ============================================================

# print("🔥 인프라 완전 삭제 시작...")
# print("⚠️ 경고: CloudFormation 스택, Cognito, Secrets Manager가 삭제됩니다!")
# print()

# # ------------------------------------------------------------
# # 11-1: Secrets Manager 삭제
# # ------------------------------------------------------------
# print("🔐 Secrets Manager 삭제 중...")
# try:
#     secrets_client.delete_secret(
#         SecretId="ecommerce_customer_support_agent",
#         ForceDeleteWithoutRecovery=True
#     )
#     print("  ✅ 삭제: ecommerce_customer_support_agent")
# except secrets_client.exceptions.ResourceNotFoundException:
#     print("  ⚠️ 없음: ecommerce_customer_support_agent")
# except Exception as e:
#     print(f"  ❌ 실패: {e}")

# # ------------------------------------------------------------
# # 11-2: 인프라 관련 SSM 파라미터 삭제
# # ------------------------------------------------------------
# print("\n⚙️ 인프라 SSM 파라미터 삭제 중...")
# infra_params = [
#     "/app/ecommerce/agentcore/runtime_iam_role",
#     "/app/ecommerce/agentcore/cognito_user_pool_id",
#     "/app/ecommerce/agentcore/cognito_client_id",
# ]

# for param_name in infra_params:
#     try:
#         ssm_client.delete_parameter(Name=param_name)
#         print(f"  ✅ 삭제: {param_name}")
#     except ssm_client.exceptions.ParameterNotFound:
#         print(f"  ⚠️ 없음: {param_name}")
#     except Exception as e:
#         print(f"  ❌ 실패: {param_name} - {e}")

# # ------------------------------------------------------------
# # 11-3: CloudFormation 스택 삭제
# # ------------------------------------------------------------
# print("\n🏗️ CloudFormation 스택 삭제 중...")
# cfn_stacks = [
#     "EcommerceCustomerSupportStackInfra",
#     "EcommerceCustomerSupportStackCognito",
# ]

# for stack_name in cfn_stacks:
#     try:
#         cloudformation_client.delete_stack(StackName=stack_name)
#         print(f"  ✅ 삭제 요청: {stack_name}")
#     except Exception as e:
#         print(f"  ❌ 실패: {stack_name} - {e}")

# # 스택 삭제 완료 대기
# print("\n⏳ CloudFormation 스택 삭제 대기 중...")
# for stack_name in cfn_stacks:
#     try:
#         waiter = cloudformation_client.get_waiter('stack_delete_complete')
#         waiter.wait(
#             StackName=stack_name,
#             WaiterConfig={'Delay': 10, 'MaxAttempts': 60}
#         )
#         print(f"  ✅ 삭제 완료: {stack_name}")
#     except Exception as e:
#         print(f"  ⚠️ 대기 중 오류 (이미 삭제됨?): {stack_name} - {e}")

# # ------------------------------------------------------------
# # 11-4: CodeBuild IAM 역할 삭제 (AgentCore가 생성한 것)
# # ------------------------------------------------------------
# print("\n🔑 CodeBuild IAM 역할 삭제 중...")
# try:
#     paginator = iam_client.get_paginator('list_roles')
#     for page in paginator.paginate():
#         for role in page['Roles']:
#             role_name = role['RoleName']
#             if 'AmazonBedrockAgentCoreSDKCodeBuild' in role_name:
#                 print(f"  📋 발견: {role_name}")
#                 try:
#                     # 연결된 정책 분리
#                     attached = iam_client.list_attached_role_policies(RoleName=role_name)
#                     for policy in attached['AttachedPolicies']:
#                         iam_client.detach_role_policy(
#                             RoleName=role_name,
#                             PolicyArn=policy['PolicyArn']
#                         )
#                     # 인라인 정책 삭제
#                     inline = iam_client.list_role_policies(RoleName=role_name)
#                     for policy_name in inline['PolicyNames']:
#                         iam_client.delete_role_policy(
#                             RoleName=role_name,
#                             PolicyName=policy_name
#                         )
#                     # 역할 삭제
#                     iam_client.delete_role(RoleName=role_name)
#                     print(f"    ✅ 삭제 완료: {role_name}")
#                 except Exception as e:
#                     print(f"    ❌ 삭제 실패: {e}")
# except Exception as e:
#     print(f"  ❌ IAM 역할 조회 실패: {e}")

# # ------------------------------------------------------------
# # 11-5: S3 버킷 삭제 (CodeBuild 소스용)
# # ------------------------------------------------------------
# print("\n🪣 S3 버킷 삭제 중...")
# s3_client = session.client('s3')
# s3_resource = session.resource('s3')

# bucket_patterns = [
#     f"bedrock-agentcore-codebuild-sources-{ACCOUNT_ID}-{REGION}",
# ]

# for bucket_name in bucket_patterns:
#     try:
#         bucket = s3_resource.Bucket(bucket_name)
#         # 버킷 내 모든 객체 삭제
#         bucket.objects.all().delete()
#         bucket.object_versions.all().delete()
#         # 버킷 삭제
#         s3_client.delete_bucket(Bucket=bucket_name)
#         print(f"  ✅ 삭제: {bucket_name}")
#     except s3_client.exceptions.NoSuchBucket:
#         print(f"  ⚠️ 없음: {bucket_name}")
#     except Exception as e:
#         print(f"  ❌ 실패: {bucket_name} - {e}")

# # ------------------------------------------------------------
# # 완료
# # ------------------------------------------------------------
# print("\n" + "=" * 70)
# print("🔥 인프라 완전 삭제 완료!")
# print("=" * 70)
# print("\n삭제된 리소스:")
# print("   - Secrets Manager: ecommerce_customer_support_agent")
# print("   - SSM 파라미터: 인프라 관련 전체")
# print("   - CloudFormation: Cognito, IAM 스택")
# print("   - IAM: CodeBuild 역할")
# print("   - S3: CodeBuild 소스 버킷")
# print("\n🎯 처음부터 다시 시작하려면:")
# print("   ./setup/deploy_infra.sh 실행 후 Lab 01부터 시작")